In [ ]:
# get calibration by each type of token

sql_funcs = ["SELECT", "WHERE", "GROUP BY", "HAVING", "ORDER BY", "LIMIT", "JOIN", "INTERSECT", "EXCEPT", "UNION", "NOT IN", "OR", "AND", "EXISTS", "LIKE"]

def get_token_type(tokens, probs):
    






